In [1]:
import torch
from torch import nn
from torch.nn import functional as F

In [2]:
num_samples = 2
num_inputs = 20
num_hiddens = 256
num_outputs = 10

net = nn.Sequential(
    nn.Linear(num_inputs, num_hiddens),
    nn.ReLU(),
    nn.Linear(num_hiddens, num_outputs)
)

X = torch.rand(num_samples, num_inputs)
net(X).shape

torch.Size([2, 10])

In [3]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(num_inputs, num_hiddens)
        self.out = nn.Linear(num_hiddens, num_outputs)

    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

In [4]:
net = MLP()
net(X).shape

torch.Size([2, 10])

In [5]:
class MySequential(nn.Module):
    def __init__(self, *args):  # args는 튜플
        super().__init__()
        for idx, module in enumerate(args):
            self.add_module(str(idx), module)  # self._modules[str(idx)] = module

    def forward(self, X):
        for module in self.children():
            X = module(X)
        return X

In [6]:
net = MySequential(
    nn.Linear(num_inputs, num_hiddens),
    nn.ReLU(),
    nn.Linear(num_hiddens, num_outputs)
)

In [7]:
net(X).shape

torch.Size([2, 10])

In [8]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(20, 20)
        self.rand_weight = torch.rand((20, 20))  # parameter 아님 → update 되지 않음

    def forward(self, X):
        X = self.linear(X)
        X = F.relu(X@self.rand_weight + 1)
        X = self.linear(X)  # layer 재사용

        # control flow → 데이터에 따라 autograd 그래프 달라짐 (=동적)
        while X.abs().sum() > 1:
            X /= 2

        return X.sum()

In [9]:
net = FixedHiddenMLP()
net(X)

tensor(0.1357, grad_fn=<SumBackward0>)